In [ ]:
#https://web.archive.org/web/20200917010635/https://medium.com/@mommermiscience/dealing-with-geospatial-raster-data-in-python-with-rasterio-775e5ba0c9f5
#https://web.archive.org/web/20210108101658/https://towardsdatascience.com/sentinel-2-image-clustering-in-python-58f7f2c8a7f6?gi=e7128e2cf8b2
#https://learn.microsoft.com/en-us/azure/developer/dev-tunnels/get-started?tabs=linux
#https://web.archive.org/web/20210108101658/https://towardsdatascience.com/sentinel-2-image-clustering-in-python-58f7f2c8a7f6?gi=e7128e2cf8b2

#https://web.archive.org/web/20200917010635/https://medium.com/@mommermiscience/dealing-with-geospatial-raster-data-in-python-with-rasterio-775e5ba0c9f5

In [1]:
import numpy as np
import rasterio
import pandas as pd

from shapely.geometry import Point
import geopandas as gpd
from pyproj import Proj, Transformer

In [2]:
ruta_denue_csv = "../input_data/denue.csv"
try:
    df = pd.read_csv(ruta_denue_csv, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(ruta_denue_csv, encoding="latin1")
df.head()

/tmp/ipykernel_611/2985616568.py:5: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_denue_csv, encoding="latin1")


,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,...,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
0,6862345,01001115119000031001000000U2,AGROPECUARIA CHARCOS DE QUEZADA,AGROPECUARIA CHARCOS DE QUEZADA SOCIEDAD DE PR...,115119,Otros servicios relacionados con la agricultura,6 a 10 personas,AVENIDA,UNIVERSIDAD,CALLE,...,Aguascalientes,0233,9,4492853716,NaN,NaN,Fijo,21.906992,-102.309807,2019-04
1,6905212,01001115119000041000000000U1,AGROPECUARIO LOS ALAMITOS,NaN,115119,Otros servicios relacionados con la agricultura,0 a 5 personas,CALLE,J. ENCARNACION PAYEN,CALLE,...,Aguascalientes,0322,18,4499960472,NaN,NaN,Fijo,21.889926,-102.314009,2019-11
2,6166248,01007112512000012001000000U8,CENTRO ACUÍCOLA PABELLON DE HIDALGO,SAGARPA,112512,"Piscicultura y otra acuicultura, excepto camar...",0 a 5 personas,CALLE,NINGUNO,CALLE,...,SEMARNAT [Centro Acuícola],0078,800,NaN,NaN,WWW.GOB.MX,Fijo,22.167778,-102.345556,2010-07
3,6166233,01010112512000011000000000U9,G. ACUÍCOLA SANTA ROSA,NaN,112512,"Piscicultura y otra acuicultura, excepto camar...",6 a 10 personas,CARRETERA,SECTOR DE PRODUCCION N0 1 TERREMOTO EL LLANO A...,CARRETERA,...,Santa Rosa [Granja Acuícola],0069,800,NaN,NaN,NaN,Fijo,21.956298,-101.997312,2014-12
4,6166234,01003112512000031001000000U7,GRANJA ACUICOLA ARROYO DE LOS CABALLOS,NaN,112512,"Piscicultura y otra acuicultura, excepto camar...",0 a 5 personas,PROLONGACION,CUAUHTUMOC,CARRETERA,...,Arroyo Ojocalientillo,0100,2,4492741517,MARTINSAU2720@HOTMAIL.COM,NaN,Fijo,21.837255,-102.710931,2014-12


In [3]:
df_limpio = df[["codigo_act", "latitud", "longitud"]].copy()
df_limpio["codigo_act"] = df_limpio["codigo_act"].astype(str).str[:2]

In [4]:
print("Valores nulos por columna:")
df_limpio.isnull().sum()

Valores nulos por columna:


codigo_act    0
latitud       0
longitud      0
dtype: int64

In [5]:
df_limpio.dtypes

codigo_act     object
latitud       float64
longitud      float64
dtype: object

In [13]:
sentinel_files = [str(x) if x >= 10 else "0"+str(x)  for x in range(1,13)]
sentinel_files.append("8A")
sentinel_base = "../input_data/sentinel/T13QGE_20231208T172711_B"
sentinel_extension = ".jp2"

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '8A']

In [14]:
for sentinel_file in sentinel_files:
    print(sentinel_file)
    #ruta_landsat_tiff = "../input_data/sentinel/T13QGE_20231208T172711_B02.jp2"
    current_sentinel = sentinel_base + sentinel_file + sentinel_extension
    with rasterio.open(current_sentinel, driver="JP2OpenJPEG") as src:
        srs_raster = src.crs.to_proj4()

        # Crear columna "geometry" utilizando latitud y longitud
        df_limpio["geometry_"+sentinel_file] = [Point(xy) for xy in zip(df_limpio["longitud"], df_limpio["latitud"])]

        # Convertir el DataFrame de pandas en un GeoDataFrame de GeoPandas
        gdf = gpd.GeoDataFrame(df_limpio, geometry="geometry")

01
02
03
04
05
06
07
08
09
10
11
12
8A


In [ ]:
with rasterio.open(ruta_landsat_tiff, driver="JP2OpenJPEG") as src:
    srs_raster = src.crs.to_proj4()

In [ ]:
# Crear columna "geometry" utilizando latitud y longitud
df_limpio["geometry"] = [Point(xy) for xy in zip(df_limpio["longitud"], df_limpio["latitud"])]

# Convertir el DataFrame de pandas en un GeoDataFrame de GeoPandas
gdf = gpd.GeoDataFrame(df_limpio, geometry="geometry")
srs_gdf = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
transformador = Transformer.from_proj(Proj(srs_gdf), Proj(srs_raster))
gdf["geometry2"] = gdf["geometry"].apply(lambda geom: Point(transformador.transform(geom.x, geom.y)))
gdf.head()

In [ ]:
# Abre el archivo GeoTIFF con rasterio
with rasterio.open(ruta_landsat_tiff, driver="JP2OpenJPEG") as src:
    # Itera sobre los puntos del GeoDataFrame y obtiene los valores de los pixeles correspondientes
    for index, row in gdf.iterrows():
        gdf.loc[index, ["col_pixel", "row_pixel"]] = src.index(row["geometry2"].x, row["geometry2"].y)


In [ ]:
gdf

In [ ]:
gdf_cp = gdf.copy()

In [ ]:
dups = gdf_cp[gdf_cp.duplicated(subset=["col_pixel", "row_pixel"], keep=False) == True]

In [ ]:
pd.set_option('display.max_rows', None)
dups = dups.groupby(["col_pixel", "row_pixel"],as_index=False).size()

In [ ]:
dups.sort_values(["size"], ascending=False)

In [ ]:
dups["size"].sum()

In [ ]:
gdf.iloc[[916]]

In [ ]:
gdf.query("col_pixel == 1292 and row_pixel == 1321")